In [1]:
import time
from openai import OpenAI, RateLimitError

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="sk-Cbg9C13UKtSQJE7HXdMRRktxVNmBmM20UN8f8WOmyCy6oq0r",
    base_url="https://api.chatanywhere.tech/v1"
)

def gpt_35_api(messages: list):
    completion = client.chat.completions.create(model="gpt-3.5-turbo", messages=messages)
    return completion.choices[0].message.content

def gpt_35_api_with_retry(messages, max_retries=5, retry_delay=60):
    for attempt in range(max_retries):
        try:
            # 尝试调用API
            result = gpt_35_api(messages).strip().split('\n')
            return result
        except RateLimitError:
            print(f"Rate limit exceeded, retrying in {retry_delay} seconds...")
            time.sleep(retry_delay)
    raise Exception("API request failed after maximum number of retries.")

start = time.time()
results = []
with open('car_reports.txt', 'r') as file:
    reports = file.read()
    lines = reports.split('\n')
    prompt = ("Can you analyze this text and format your response as follows? \n"
        "车辆型号: [Vehicle Model],\n"
        "引擎排量: [Engine Displacement],\n"
        "生产年份: [Production Year],\n"
        "车辆排量: [Vehicle Fuel Tank Capacity],\n"
        "车辆油耗: [Fuel Consumption],\n"
        "变速箱类型: [Transmission Type],\n")
    for line in lines:
        report = line.strip()
        content = (f"The text is:{report}\n"
                   f"{prompt}")
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": content}
            ]
        result = gpt_35_api_with_retry(messages).strip().split('\n')
        print(result)
        results.append(result)
end = time.time()
print(f"Time taken: {end-start}")     

Rate limit exceeded, retrying in 60 seconds...
